In [3]:
import inspect, pandas as pd, os

In [4]:
from IPython.core.display import display, HTML

In [5]:
projects_path = '../projects.csv'

In [6]:
os.listdir('../img/')

['github.jpg',
 'streamlit_webapp.jpg',
 'tunasoma.jpg',
 'early_reading.jpg',
 'rongzi.jpg',
 'Steven_headshot_500px.jpg',
 'youtube.jpg',
 'predwikt.jpg']

In [14]:
css = """
/*!
 * Table styles for portfolio
 */

table, th, td {border: 2px solid gray;}
table {width= 90%}

.link_img {
  display: block;
  margin-left: auto;
  margin-right: auto;
  width:6em
}
"""

with open('../styles.css', 'w') as f:
    f.write(css)

In [19]:
"""
Generate an html file for my portfolio site.
    Pull text and links from /projects.csv
    `git pull` all listed repos
    Pull images and data from local (updated) repos etc.
    (Over)Write the html file
    (Over)Write the images and data
    """
def sp2(s:str) -> str:
    "Prepend two spaces before each line of a multiline string."
    lines = s.split('\n')
    for i, line in enumerate(lines):
        lines[i] = '  ' + line
    return '\n'.join(lines)
def is_too_short(y):
    x = True
    try:
        if len(y) > 0:
            x = False
    except Exception as e:
        pass
    return x

def complete(html=''): 
    "Return a string including the <html> tags and everything outside them."
    x = (
        f"<!DOCTYPE html>\n"
        f"<html>\n"
        f"{html}\n"
        f"</html>"
    )
    return x
def html(head='', body=''):
    "Return a string organizing everything inside the <html> tags."
    x = (
        f'<head>\n'
        f'{sp2(head)}\n'  # background color should be rgb(213, 218, 229);
        f'</head>\n'
        f'<body>\n'
        f'{sp2(body)}\n'
        f'</body>'
    )
    return x
def head(title=''):
    "Return a string organizing everything inside the <head> tags."
    x = (
        f'<title>{title}</title>\n'
        f'<link type="text/css" rel="stylesheet" href="bootstrap.css">\n'
        f'<link rel="stylesheet" href="styles.css">'
    )
    return x
def body(headers='', tables='', footers=''):
    "Return a string organizing everything inside the <body> tags."
    x = (
        f'{headers}\n'
        f'<br>\n'
        f'{tables}\n'
        f'<br>\n'
        f'{footers}'
    )
    return x
def footers():
    "Return a string organizing the footers, the bottom of the page <body>."
    x = f''
    return x
def headers(maintitle='', subtitle=''):
    "Return a string organizing the headers, the top of the page <body>."
    x = (
        f'<h1 style="text-align:center">{maintitle}</h1>\n'
        f'<h3 style="text-align:center">{subtitle}</h3>'
    )
    return x
def title(maintitle='', subtitle=''):
    "Return a string for the page title."
    if subtitle:
        x = f"{maintitle}: {subtitle}"
    else:
        x = maintitle
    return x
def tables(prjs):
    """
        Return a complete string for all the tables,
        comprising the middle of the page <body>.
        """
    x = ''
    for i in prjs.index:
        x += table(prjs.loc[i]) + '\n'
    return x
def table(prj):
    "Return a complete string for a single table for a portfolio project."
    # table background color should be white
    # borders may match page background at rgb(213, 218, 229);
    main_link = prj.loc["link_links"].split(',')[0]
    x = (
        f'<div class="d-flex justify-content-center">\n'
        f'<table width="90%">\n'
        f'  <tr class="table-default">\n'
        f'    <th width="50%" style="table-active ps-4">\n'
        f'      <h3> {prj.loc["prj_name"]}\n'
        f'      </h3>\n'
        f'    </th>\n'
        f'    <th rowspan="2" style="text-align: center;" class="p-2">\n'
        f'      <a href="{main_link}">\n'
        f'        <img src= "{prj.loc["img_path"]}" style="height:15em">\n'
        f'      </a>\n'
        f'    </th>\n'
        f'  </tr>\n'
        f'  <tr>\n'
        f'    <td width="50%" class="p-2 table-default">\n'
        f'{blurb_box(prj)}'
        f'    </td>\n'
        f'  </tr>\n'
        f'</table>\n'
        f'</div>\n'
        f'<br><br>\n'
    )
    return x

def get_dict_of_link_html(prj):
    link_html = {}

    # make sure there are image links
    if is_too_short(prj.link_links):
        return ''
    # get lists of hyperlinks and project-names
    links = prj.link_links.split(',')
    names = prj.link_names.split(',')

    # get list of image file stems
    img_stems = []
    for n in os.listdir('../img/'):
        split_n = n.split('.')
        assert split_n[-1] == 'jpg'  # only jpgs allowed in image folder for convenience
        assert len(split_n) == 2     # no additional periods allowed in filenames for clarity
        img_stems.append(split_n[0])

    # get list of html lines for hyperlinks
    for n, l in zip(names, links):
        if n in img_stems:
            p = f'img/{n}.jpg' # final image path
            x = f'<a href="{l}"><img src="{p}" class="link_img"></a>' # image html with hyperlink
            link_html[n] = x
        else:
            x = f'<a href="{l}">{n}</a>' # link name only with hyperlink
            link_html[n] = x
    return (link_html, len(links))

def get_link_html(prj):
    link_html, n_links = get_dict_of_link_html(prj) # list
    link_html = (
        f'<div style="column-count:{n_links}">' +
        '\n        <div>' +
        '</div>\n        <div>'.join(link_html.values()) + 
        '</div>\n      </div>' 
    )# multiline html string
    return link_html

def blurb_box(prj):
    link_html = get_link_html(prj)
    
    x = (
        f'      {prj.loc["blurb1"]}\n'
        f'      <br><br>\n'
        f'      <details>\n'
        f'        <summary><i>show/hide more details</i></summary>\n'
        f'        <br>\n'
        f'        <i><p>{prj.loc["blurb2"]}</p></i>\n'
        f'      </details>\n'
        f'      <br>\n'
        f'      {link_html}\n'
    )
    
    return x

def get_values(values:dict):
    """
    Run all the functions in the list in order, 
        feeding their content to each other.
    """
    func_list = [tables, title, headers, footers, body, head, html, complete]
    func_dict = {i.__name__:i for i in func_list}
    for f in func_dict:
        args_names = inspect.getfullargspec(func_dict[f]).args
        args_values = {n:values[n] for n in args_names}
        values[f] = func_dict[f](**args_values)
    return values

prjs = pd.read_csv(projects_path, sep=';')

input_values = {
    'maintitle': "Steven Bhardwaj"
    ,'subtitle': "Full-Stack Data Science"
    ,'prjs': prjs
}


In [20]:
values = get_values(input_values)
html_output = values['complete']
with open('../index.html', 'w') as f:
    f.write(html_output)